# Notebook 03: LlamaStack Core Features

## 🎯 What is This Notebook About?

Welcome to Notebook 03! In this notebook, we'll explore **LlamaStack's core capabilities** - the building blocks that make powerful agents possible.

**What we'll learn:**
1. **Simple Chat** - Basic LLM interactions
2. **RAG (Retrieval Augmented Generation)** - Enhancing LLMs with external knowledge

**Why this matters:**
- Understanding these features helps you build better agents
- Each feature solves a specific problem
- Combining features creates powerful solutions
- This knowledge prepares you for advanced agent development

---

## 📚 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand LlamaStack's core capabilities
- ✅ Know when to use each feature
- ✅ See how features work independently
- ✅ Be ready to explore MCP tools (Notebook 04), Safety (Notebook 05), and Evaluation (Notebook 06)

---

## ⚙️ Prerequisites

- LlamaStack server running (see Module README)
- Ollama running with llama3.2:3b model
- Python environment with dependencies installed

---

## 🔧 Setup

Let's start by connecting to LlamaStack and verifying everything is working.


In [ ]:
# Import required libraries
import os
from llama_stack_client import LlamaStackClient
from termcolor import cprint

# Configuration
llamastack_url = os.getenv("LLAMA_STACK_URL", "http://localhost:8321")
model = os.getenv("LLAMA_MODEL", "ollama/llama3.2:3b")

print(f"📡 LlamaStack URL: {llamastack_url}")
print(f"🤖 Model: {model}")

# Initialize LlamaStack client
client = LlamaStackClient(base_url=llamastack_url)

# Verify connection
try:
    models = client.models.list()
    print(f"\n✅ Connected to LlamaStack")
    print(f"   Available models: {len(models)}")
except Exception as e:
    print(f"\n❌ Cannot connect to LlamaStack: {e}")
    print("   Please ensure LlamaStack is running:")
    print("   python scripts/start_llama_stack.py")
    raise


---

## Part 1: Simple Chat

### What is Chat?

**Chat** is the most basic way to interact with an LLM. It's a conversation where you send messages and receive responses.

**Key Concepts:**
- **Message Types**: System (instructions), User (questions), Assistant (responses)
- **Streaming vs Non-streaming**: Get responses as they're generated or wait for complete response
- **Conversation Context**: LLM remembers previous messages in the conversation

**When to use Chat:**
- Simple Q&A
- Text generation
- Basic reasoning tasks
- When you don't need external knowledge or tools

---

### Hands-on: Basic Chat Completion

Let's start with the simplest example - a single question and answer.


In [ ]:
# Example 1: Basic chat completion
print("=" * 60)
print("Example 1: Basic Chat Completion")
print("=" * 60)

# Create a simple chat completion
response = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": "What is artificial intelligence in one sentence?"
        }
    ]
)

# Extract and display the response
answer = response.choices[0].message.content
print(f"\n📝 Question: What is artificial intelligence in one sentence?")
print(f"\n🤖 Answer:\n{answer}\n")


### System Prompts

**System prompts** are instructions that guide the LLM's behavior. They set the "personality" and "role" of the assistant.

**Why use system prompts:**
- Define the assistant's role (e.g., "You are a helpful IT operations assistant")
- Set behavior guidelines
- Provide context about the domain
- Ensure consistent responses


In [ ]:
# Example 2: Chat with system prompt
print("=" * 60)
print("Example 2: Chat with System Prompt")
print("=" * 60)

response = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful IT operations assistant. You provide clear, concise answers about IT infrastructure and operations."
        },
        {
            "role": "user",
            "content": "What should I check if a web server is not responding?"
        }
    ]
)

answer = response.choices[0].message.content
print(f"\n📝 Question: What should I check if a web server is not responding?")
print(f"\n🤖 Answer (with IT operations context):\n{answer}\n")


### Multi-turn Conversations

**Multi-turn conversations** maintain context across multiple exchanges. The LLM remembers previous messages in the conversation.

**Why this matters:**
- Natural conversation flow
- Can refer back to earlier topics
- Builds on previous context
- More human-like interaction


In [ ]:
# Example 3: Multi-turn conversation
print("=" * 60)
print("Example 3: Multi-turn Conversation")
print("=" * 60)

# First turn
messages = [
    {
        "role": "user",
        "content": "I'm setting up a new database server. What should I consider?"
    }
]

response1 = client.chat.completions.create(
    model=model,
    messages=messages
)

answer1 = response1.choices[0].message.content
print(f"\n📝 Turn 1 - Question: I'm setting up a new database server. What should I consider?")
print(f"\n🤖 Answer:\n{answer1[:200]}...\n")

# Second turn - add previous messages to maintain context
messages.append({
    "role": "assistant",
    "content": answer1
})
messages.append({
    "role": "user",
    "content": "What about security specifically?"
})

response2 = client.chat.completions.create(
    model=model,
    messages=messages
)

answer2 = response2.choices[0].message.content
print(f"\n📝 Turn 2 - Question: What about security specifically?")
print(f"   (Note: The assistant knows we're talking about database servers)\n")
print(f"🤖 Answer:\n{answer2[:200]}...\n")


### Streaming Responses

**Streaming** allows you to receive the response as it's being generated, token by token. This provides:
- Faster perceived response time
- Real-time feedback
- Better user experience

**When to use streaming:**
- Long responses
- Interactive applications
- When you want immediate feedback


In [ ]:
# Example 4: Streaming response
print("=" * 60)
print("Example 4: Streaming Response")
print("=" * 60)

print(f"\n📝 Question: Explain what RAG (Retrieval Augmented Generation) is.\n")
print("🤖 Answer (streaming):\n")

# Create streaming completion
stream = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": "Explain what RAG (Retrieval Augmented Generation) is in 2-3 sentences."
        }
    ],
    stream=True  # Enable streaming
)

# Process stream chunk by chunk
full_response = ""
for chunk in stream:
    if chunk.choices[0].delta.content:
        content = chunk.choices[0].delta.content
        print(content, end="", flush=True)
        full_response += content

print("\n\n✅ Streaming complete!")


---

## Part 2: RAG (Retrieval Augmented Generation)

### What is RAG?

**RAG** enhances LLMs with external knowledge by:
1. **Storing documents** in a vector database (vector store)
2. **Searching** for relevant context when answering questions
3. **Augmenting** the LLM's prompt with retrieved context

**Why RAG matters:**
- LLMs have training data cutoff dates
- Can't access private/internal documents
- RAG provides up-to-date, domain-specific knowledge
- Improves accuracy for specialized topics

**When to use RAG:**
- Need access to specific documents
- Domain-specific knowledge required
- Private/internal information
- Up-to-date information needed

---

### Hands-on: Creating a Vector Store

Let's create a vector store and add some IT operations documentation.


In [ ]:
# Example 1: Create a vector store
print("=" * 60)
print("Example 1: Creating a Vector Store")
print("=" * 60)

# Sample IT operations documentation
it_docs = [
    {
        "id": "doc1",
        "content": "To restart a web server, use: systemctl restart nginx. Check status with: systemctl status nginx."
    },
    {
        "id": "doc2",
        "content": "High CPU usage troubleshooting: 1) Check top processes with 'top' or 'htop', 2) Identify CPU-intensive processes, 3) Check for runaway processes or infinite loops."
    },
    {
        "id": "doc3",
        "content": "Database connection issues: Check firewall rules, verify credentials, ensure database service is running, check network connectivity with 'telnet hostname port'."
    },
    {
        "id": "doc4",
        "content": "Disk space issues: Use 'df -h' to check disk usage, find large files with 'du -sh /*', clean logs with 'journalctl --vacuum-time=7d'."
    },
    {
        "id": "doc5",
        "content": "Service monitoring: Use 'systemctl list-units --type=service' to list all services, 'systemctl is-active servicename' to check status, set up monitoring with Prometheus or Nagios."
    }
]

print(f"\n📚 Sample IT Operations Documentation:")
for doc in it_docs:
    print(f"   - {doc['id']}: {doc['content'][:60]}...")

print("\n💡 These documents will be stored in a vector store for retrieval.")


In [ ]:
# Create vector store using LlamaStack
print("\n" + "=" * 60)
print("Creating Vector Store")
print("=" * 60)

from io import BytesIO

# Step 1: Create files from text content
print(f"\n📝 Creating files from {len(it_docs)} documents...")
file_ids = []

for i, doc in enumerate(it_docs, 1):
    # Create a file-like object from the document content
    file_content = BytesIO(doc["content"].encode('utf-8'))
    file_name = f"doc_{i}.txt"
    
    # Upload file to LlamaStack
    # The API expects a tuple: (filename, file_content, content_type)
    file_obj = (file_name, file_content, 'text/plain')
    
    uploaded_file = client.files.create(
        file=file_obj,
        purpose="assistants"
    )
    file_ids.append(uploaded_file.id)
    print(f"   ✅ Uploaded {file_name} (ID: {uploaded_file.id})")

print(f"\n✅ Created {len(file_ids)} files")

# Step 2: Create vector store with files
print(f"\n📦 Creating vector store...")
vector_store = client.vector_stores.create(
    name="it-operations-docs",
    file_ids=file_ids,
    metadata={"description": "IT operations documentation and troubleshooting guides"}
)

print(f"\n✅ Vector store created!")
print(f"   Name: {vector_store.name}")
print(f"   ID: {vector_store.id}")
print(f"   Files: {len(file_ids)}")

# Step 3: Wait for files to be processed (vector stores need time to index files)
print(f"\n⏳ Waiting for files to be processed and indexed...")
import time

max_wait = 30  # Maximum wait time in seconds
wait_interval = 2  # Check every 2 seconds
elapsed = 0

while elapsed < max_wait:
    # Check vector store status
    vs_status = client.vector_stores.retrieve(vector_store.id)
    
    # Check if files are processed (status might be in file_counts or similar)
    if hasattr(vs_status, 'file_counts'):
        file_counts = vs_status.file_counts
        if hasattr(file_counts, 'in_progress') and file_counts.in_progress == 0:
            print(f"   ✅ All files processed!")
            break
    elif hasattr(vs_status, 'status'):
        if vs_status.status == 'completed':
            print(f"   ✅ Vector store ready!")
            break
    
    # Check file status directly
    vs_files = client.vector_stores.files.list(vector_store.id)
    if hasattr(vs_files, 'data'):
        processed = sum(1 for f in vs_files.data if hasattr(f, 'status') and f.status == 'completed')
        if processed == len(file_ids):
            print(f"   ✅ All {processed} files processed!")
            break
    
    print(f"   ⏳ Waiting... ({elapsed}s/{max_wait}s)", end='\r')
    time.sleep(wait_interval)
    elapsed += wait_interval

if elapsed >= max_wait:
    print(f"\n   ⚠️  Timeout waiting for processing. Files may still be indexing.")
    print(f"   💡 You can proceed, but search results may be incomplete initially.")

print(f"\n💡 The vector store is ready for semantic search!")


### Searching for Relevant Context

Once documents are in the vector store, we can search for relevant context based on semantic similarity (meaning, not just keywords).

**How it works:**
1. Convert query to embedding (vector representation)
2. Compare with document embeddings
3. Return most similar documents
4. Use retrieved documents as context for LLM


In [ ]:
# Example 2: Search for relevant context using LlamaStack API
print("=" * 60)
print("Example 2: Searching Vector Store")
print("=" * 60)

query = "How do I restart a web server?"
print(f"\n🔍 Query: {query}\n")

# Search the vector store using LlamaStack API
search_results = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query=query,
    max_num_results=2
)

print("📚 Retrieved Documents (from vector store):")
print(f"   Found {len(search_results.data)} results\n")

if len(search_results.data) == 0:
    print("   ⚠️  No results found. This might mean:")
    print("      - Files are still being processed/indexed")
    print("      - Try waiting a few seconds and searching again")
    print("      - Or check if files were added correctly to the vector store")
    print("\n   💡 For demonstration, we'll use the original documents:")
    # Fallback to original documents for demonstration
    for i, doc in enumerate(it_docs[:2], 1):
        if "restart" in doc["content"].lower() or "web server" in doc["content"].lower():
            print(f"\n   {i}. {doc['id']}:")
            print(f"      {doc['content']}")
else:
    for i, result in enumerate(search_results.data, 1):
        print(f"   {i}. ", end="")
        # The result contains the document content and score
        if hasattr(result, 'score'):
            print(f"Score: {result.score:.3f}")
        if hasattr(result, 'content') and result.content:
            print(f"      Content: {result.content[:150]}...")
        elif hasattr(result, 'text') and result.text:
            print(f"      Text: {result.text[:150]}...")
        elif hasattr(result, 'document') and result.document:
            print(f"      Document: {str(result.document)[:150]}...")
        else:
            # Try to get any text-like attribute
            result_str = str(result)
            print(f"      Result: {result_str[:150]}...")
        print()

print("\n💡 These documents were retrieved using semantic search (embeddings).")
print("   They will be used as context for the LLM.")


### Using Retrieved Context in Chat

Now let's use the retrieved documents as context for the LLM. This is the "Augmented Generation" part of RAG.


In [ ]:
# Example 3: RAG - Using retrieved context in chat
print("=" * 60)
print("Example 3: RAG - Chat with Retrieved Context")
print("=" * 60)

query = "How do I restart a web server?"
print(f"\n📝 Question: {query}\n")

# Search the vector store for relevant context
search_results = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query=query,
    max_num_results=2
)

# Build context from retrieved documents
context_parts = []
for i, result in enumerate(search_results.results, 1):
    # Extract content from result
    if hasattr(result, 'content') and result.content:
        content = result.content
    elif hasattr(result, 'text') and result.text:
        content = result.text
    else:
        # Try to get content from file if available
        content = f"Document {i} (score: {result.score:.3f})"
    
    context_parts.append(f"Document {i}:\n{content}")

context = "\n\n".join(context_parts)

# Create prompt with context
prompt = f"""Use the following IT operations documentation to answer the question.

Documentation:
{context}

Question: {query}

Answer based on the documentation provided:"""

print(f"📚 Context Retrieved from Vector Store:\n{context[:300]}...\n")

# Get response with context
response = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "system",
            "content": "You are a helpful IT operations assistant. Answer questions based on the provided documentation."
        },
        {
            "role": "user",
            "content": prompt
        }
    ]
)

answer = response.choices[0].message.content
print(f"🤖 Answer (with RAG context):\n{answer}\n")
print("✅ Notice how the answer uses the specific documentation retrieved from the vector store!")


---

## Summary

### When to Use Each Feature

**Simple Chat:**
- ✅ Basic Q&A
- ✅ Text generation
- ✅ Simple reasoning
- ❌ Don't use when you need external knowledge or tools

**RAG:**
- ✅ Need access to specific documents
- ✅ Domain-specific knowledge required
- ✅ Private/internal information
- ❌ Don't use for general knowledge questions

---

### How Features Complement Each Other

**Powerful combinations:**
- **Chat + RAG**: Answer questions with domain knowledge
- **Chat + MCP**: Answer questions and take actions (see Notebook 04)
- **RAG + MCP**: Use knowledge to make informed actions (see Notebook 04)
- **All + Safety**: Production-ready agent with safety checks (see Notebook 05)
- **All + Evaluation**: Measured, safe, powerful agent (see Notebook 06)

---

### Next Steps: Exploring More Features

In the next notebooks, we'll explore:
- **Notebook 04**: MCP (Model Context Protocol) - External tool integration
- **Notebook 05**: Safety - Content moderation and safety shields
- **Notebook 06**: Evaluation - Measuring AI performance

**Ready to explore more features?** Let's move to Notebook 04!
